In [1]:
import gc

!python --version

Python 3.5.3 :: Anaconda 4.4.0 (64-bit)


In [2]:
RESULTS_DIR = "./results/session-01-KagglePlanetAmazon/"
DATA_DIR = "../../Data/kaggle-planet-amazon/"

!head ../../Data/kaggle-planet-amazon/train_v2.csv

image_name,tags
train_0,haze primary
train_1,agriculture clear primary water
train_2,clear primary
train_3,clear primary
train_4,agriculture clear habitation primary road
train_5,haze primary water
train_6,agriculture clear cultivation primary water
train_7,haze primary
train_8,agriculture clear cultivation primary


In [3]:
import pandas as pd

In [4]:
df_train = pd.read_csv(DATA_DIR + "train_v2.csv")

# df_train["tags"] = df_train["tags"].map(lambda x: x.split(" "))

df_train.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [5]:
# binarizedLabels_df = pd.DataFrame(binarizedLabels)
# binarizedLabels_df.columns = mlb.classes_
# binarizedLabels_df.head()

In [23]:
import numpy as np
import cv2
from tqdm import tqdm

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

from sklearn.metrics import fbeta_score
# from sklearn.metrics import matthews_corrcoef, hamming_loss

In [7]:
df_valid_samples = df_train.sample(frac=0.2)

df_train_samples = df_train.drop(df_valid_samples.index)

In [8]:
def PrintShape(df, df_shape):
    print("{} shape: {}".format(df, df_shape))

In [9]:
PrintShape('df_train_samples', df_train_samples.shape)
PrintShape('df_valid_samples', df_valid_samples.shape)

df_train_samples shape: (32383, 2)
df_valid_samples shape: (8096, 2)


In [ ]:
# from sklearn.preprocessing import MultiLabelBinarizer

# mlb = MultiLabelBinarizer()

# y_train = mlb.fit_transform(df_train_samples["tags"])
# y_valid = mlb.fit_transform(df_valid_samples["tags"])

# mlb.classes_

####################################

flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

In [22]:
# Params
input_size = 64
input_channels = 3

epochs = 5
batch_size = 128
learning_rate = 0.001
lr_decay = 1e-4

In [12]:
x_valid = []
y_valid = []

for f, tags in tqdm(df_valid_samples.values, miniters=1000):
    img = cv2.resize(cv2.imread(DATA_DIR + 'train-jpg/{}.jpg'.format(f)), (input_size, input_size))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1
    x_valid.append(img)
    y_valid.append(targets)

y_valid = np.array(y_valid, np.uint8)
x_valid = np.array(x_valid, np.float32) / 255.

100%|██████████| 8096/8096 [00:11<00:00, 704.13it/s]


In [14]:
x_train = []
y_train = []

for f, tags in tqdm(df_train_samples.values, miniters=1000):
    img = cv2.resize(cv2.imread(DATA_DIR + 'train-jpg/{}.jpg'.format(f)), (input_size, input_size))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1
    x_train.append(img)
    y_train.append(targets)
    img = cv2.flip(img, 0)  # flip vertically
    x_train.append(img)
    y_train.append(targets)
    img = cv2.flip(img, 1)  # flip horizontally
    x_train.append(img)
    y_train.append(targets)
    img = cv2.flip(img, 0)  # flip vertically
    x_train.append(img)
    y_train.append(targets)

y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float32) / 255.

100%|██████████| 32383/32383 [00:46<00:00, 696.68it/s]


In [16]:
df_test_samples = pd.read_csv(DATA_DIR + 'sample_submission_v2.csv')

x_test = []

for f, tags in tqdm(df_test_samples.values, miniters=1000):
    img = cv2.resize(cv2.imread(DATA_DIR + 'test-jpg/{}.jpg'.format(f)), (input_size, input_size))
    x_test.append(img)

x_test = np.array(x_test, np.float32)

In [20]:
model = Sequential()
model.add(BatchNormalization(input_shape=(input_size, input_size, input_channels)))
model.add(Conv2D(32, kernel_size=(2, 2), padding='same', activation='relu'))
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(2, 2), padding='same', activation='relu'))
model.add(Conv2D(64, kernel_size=(2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(2, 2), padding='same', activation='relu'))
model.add(Conv2D(128, kernel_size=(2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, kernel_size=(2, 2), padding='same', activation='relu'))
model.add(Conv2D(256, kernel_size=(2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

In [63]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=5,
                           verbose=0),
             TensorBoard(log_dir='logs'),
             ModelCheckpoint('weights.h5',
                             save_best_only=True)]

opt = SGD(lr=learning_rate, decay=lr_decay)

model.compile(loss='binary_crossentropy',
              # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer=opt,
              metrics=['accuracy'])

model.fit(x_train,
          y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=2,
          callbacks=callbacks,
          validation_data=(x_valid, y_valid))

Train on 129532 samples, validate on 8096 samples
Epoch 1/5
132s - loss: 0.0959 - acc: 0.9626 - val_loss: 0.0991 - val_acc: 0.9624
Epoch 2/5
132s - loss: 0.0947 - acc: 0.9630 - val_loss: 0.0989 - val_acc: 0.9623
Epoch 3/5
132s - loss: 0.0947 - acc: 0.9629 - val_loss: 0.0987 - val_acc: 0.9624
Epoch 4/5
132s - loss: 0.0943 - acc: 0.9631 - val_loss: 0.0987 - val_acc: 0.9626
Epoch 5/5
132s - loss: 0.0943 - acc: 0.9632 - val_loss: 0.0985 - val_acc: 0.9625


In [67]:
p_test = model.predict(x_test, batch_size=batch_size, verbose=2)
p_test

array([[  1.00000000e+00,   0.00000000e+00,   1.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   1.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00,   1.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   1.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00,   1.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   1.00000000e+00],
       ..., 
       [  1.00000000e+00,   0.00000000e+00,   1.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   1.00000000e+00],
       [  1.00000000e+00,   2.15975014e-38,   1.00000000e+00, ...,
          0.00000000e+00,   2.16432707e-16,   1.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00,   1.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   1.00000000e+00]], dtype=float32)

In [66]:
p_valid = model.predict(x_valid, batch_size=batch_size)
p_valid

array([[  9.48763251e-01,   3.62713990e-07,   4.81881499e-02, ...,
          1.21221016e-03,   3.06543522e-02,   2.27194298e-02],
       [  9.98519242e-01,   2.53973074e-08,   1.91464320e-01, ...,
          9.36937460e-04,   5.72787889e-04,   2.39368752e-01],
       [  9.29065228e-01,   3.65488347e-07,   3.50466296e-02, ...,
          1.91347708e-03,   1.71075203e-02,   1.36643827e-01],
       ..., 
       [  9.99210000e-01,   1.94678961e-07,   1.15636771e-03, ...,
          8.74504403e-05,   1.98195718e-04,   3.99420183e-04],
       [  9.87671256e-01,   1.01347553e-06,   1.45826861e-01, ...,
          3.86267719e-11,   2.61703860e-02,   1.15703970e-01],
       [  2.44156674e-01,   4.11014116e-06,   8.98726523e-01, ...,
          1.20391305e-05,   2.92568523e-02,   6.83853745e-01]], dtype=float32)

In [68]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import hamming_loss

threshold = np.arange(0.1,0.9,0.1)

acc = []
accuracies = []
best_threshold = np.zeros(p_valid.shape[1])
for i in range(p_valid.shape[1]):
    y_prob = np.array(p_valid[:,i])
    for j in threshold:
        y_pred = [1 if prob>=j else 0 for prob in y_prob]
        acc.append(matthews_corrcoef(y_valid[:,i],y_pred))
    acc   = np.array(acc)
    index = np.where(acc==acc.max()) 
    accuracies.append(acc.max()) 
    best_threshold[i] = threshold[index[0][0]]
    acc = []

/home/ec2-user/src/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:516: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(var_yt * var_yp)


In [69]:
best_threshold

array([ 0.6,  0.4,  0.4,  0.2,  0.7,  0.4,  0.4,  0.4,  0.3,  0.1,  0.1,
        0.1,  0.4,  0.4,  0.1,  0.4,  0.3])

In [75]:
p_test.shape[1]

17

In [76]:
y_pred = np.array([[1 if p_test[i,j]>=best_threshold[j] else 0 for j in range(p_test.shape[1])] for i in range(p_test.shape[0])])

In [77]:
df_test_samples['tags'] = preds
df_test_samples.to_csv('submission.csv', index=False)

In [48]:
p_valid = model.predict(x_valid, batch_size=batch_size)
print(fbeta_score(y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))

y_test = []

p_test = model.predict(x_test, batch_size=batch_size, verbose=2)
y_test.append(p_test)

result = np.array(y_test[0])
result = pd.DataFrame(result, columns=labels)

preds = []

for i in tqdm(range(result.shape[0]), miniters=1000):
    a = result.ix[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

df_test_samples['tags'] = preds
df_test_samples.to_csv('submission.csv', index=False)

0.912979763217


  0%|          | 0/61191 [00:00<?, ?it/s]/home/ec2-user/src/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
100%|██████████| 61191/61191 [02:20<00:00, 434.26it/s]


In [57]:
y_test[0]

array([[ 1.,  0.,  1., ...,  0.,  0.,  1.],
       [ 1.,  0.,  1., ...,  0.,  0.,  1.],
       [ 1.,  0.,  1., ...,  0.,  0.,  1.],
       ..., 
       [ 1.,  0.,  1., ...,  0.,  0.,  1.],
       [ 1.,  0.,  1., ...,  0.,  0.,  1.],
       [ 1.,  0.,  1., ...,  0.,  0.,  1.]], dtype=float32)

In [38]:
y_test = np.array(y_test, np.uint8)

In [39]:
y_test

array([[[1, 0, 1, ..., 0, 0, 1],
        [1, 0, 1, ..., 0, 0, 1],
        [1, 0, 1, ..., 0, 0, 1],
        ..., 
        [1, 0, 1, ..., 0, 0, 1],
        [1, 0, 1, ..., 0, 0, 1],
        [1, 0, 1, ..., 0, 0, 1]]], dtype=uint8)

In [26]:
p_valid

array([[  9.57251012e-01,   5.69853000e-07,   7.38344714e-02, ...,
          1.95736997e-03,   3.50051522e-02,   3.10137104e-02],
       [  9.97661829e-01,   1.51181823e-07,   2.23692968e-01, ...,
          1.83197740e-03,   1.20442291e-03,   2.31709197e-01],
       [  9.15061414e-01,   1.01860519e-06,   6.62967637e-02, ...,
          3.15179559e-03,   2.58877594e-02,   1.98440790e-01],
       ..., 
       [  9.99349296e-01,   1.47166531e-07,   1.02706894e-03, ...,
          9.32745461e-05,   1.96420166e-04,   3.49653215e-04],
       [  9.64141905e-01,   2.34278104e-06,   1.89054370e-01, ...,
          6.42874226e-11,   8.95127803e-02,   7.64575526e-02],
       [  2.79360592e-01,   1.29797681e-05,   8.60429287e-01, ...,
          2.35701882e-05,   4.96082455e-02,   6.36332929e-01]], dtype=float32)

In [47]:
p_test

array([[ 1.,  0.,  1., ...,  0.,  0.,  1.],
       [ 1.,  0.,  1., ...,  0.,  0.,  1.],
       [ 1.,  0.,  1., ...,  0.,  0.,  1.],
       ..., 
       [ 1.,  0.,  1., ...,  0.,  0.,  1.],
       [ 1.,  0.,  1., ...,  0.,  0.,  1.],
       [ 1.,  0.,  1., ...,  0.,  0.,  1.]], dtype=float32)

In [27]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import hamming_loss

threshold = np.arange(0.1,0.9,0.1)

acc = []
accuracies = []
best_threshold = np.zeros(p_valid.shape[1])
for i in range(p_valid.shape[1]):
    y_prob = np.array(p_valid[:,i])
    for j in threshold:
        y_pred = [1 if prob>=j else 0 for prob in y_prob]
        acc.append(matthews_corrcoef(y_valid[:,i],y_pred))
    acc   = np.array(acc)
    index = np.where(acc==acc.max()) 
    accuracies.append(acc.max()) 
    best_threshold[i] = threshold[index[0][0]]
    acc = []

/home/ec2-user/src/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:516: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(var_yt * var_yp)


In [58]:
best_threshold

array([ 0.4,  0.3,  0.3,  0.2,  0.7,  0.4,  0.4,  0.5,  0.3,  0.3,  0.1,
        0.3,  0.4,  0.4,  0.1,  0.6,  0.3])

In [40]:
print(fbeta_score(y_valid, np.array(p_valid) > 0.4, beta=2, average='samples'))

0.897999527315


In [62]:
len(y_test[0])

61191

In [59]:
y_pred = np.array([[1 if p_valid[i,j]>=best_threshold[j] else 0 for j in range(y_valid.shape[1])] for i in range(len(y_test[0]))])

IndexError: index 8096 is out of bounds for axis 0 with size 8096

In [42]:
hamming_loss(y_valid,y_pred)

0.039692803999069985

In [43]:
total_correctly_predicted = len([i for i in range(len(y_valid)) if (y_valid[i]==y_pred[i]).sum() == 17])

In [44]:
total_correctly_predicted

4952

In [45]:
x_valid.shape

(8096, 64, 64, 3)

In [46]:
(total_correctly_predicted/ x_valid.shape[0]) * 100

61.16600790513834

In [36]:
gc.collect()

0

In [ ]:
!nvidia-smi

In [ ]:
# Current Mean F2-Score: 0.58